In [1]:
import pandas as pd
import re

import spacy
import numpy as np
import nltk
import string
import time
from nltk.tokenize import RegexpTokenizer

In [2]:
%%time
nlp = spacy.load("pt_core_news_lg", disable=['parser','ner','pos'])
print('spaCy Version: %s' % (spacy.__version__))

spaCy Version: 3.0.6
Wall time: 1min 16s


In [3]:
tokenizer = RegexpTokenizer(r'\w+')

In [4]:
# we'll remove the stopwords in Portuguese using Spacy
spacy_stopwords = nlp.Defaults.stop_words
print('Number of stop words: %d' % len(spacy_stopwords))
#print('Spacy stop words: %s' % spacy_stopwords)

Number of stop words: 416


In [7]:
# removing words from the stop words list
nlp.Defaults.stop_words -= {'direita', 'maiorias', 'maioria', 'grupo', 'meio', 'posição', 'apoio'}

# and adding new stop words, different verbs conjugations and pronoms
nlp.Defaults.stop_words |= {'há', 'havemos', 'havia', 'houve', 'houvemos', 'houveram', 'hemos',
                            'houvera', 'houvéramos', 'houvessem', 'houvesse', 'houvéssemos', 'heis'
                            'hão', 'haveis', 'haja', 'hajamos', 'hajam', 'hoje', 'semana', 'dia', 
                            'dias', 'anos', 'ano', 'eram', 'seja', 'será', 'sendo', 'sido', 
                            'seremos', 'serão', 'sejam', 'seríamos', 'seriam', 'estamos', 
                            'haver', 'houver', 'haverá', 'apesar', 'entanto', 'aliás', 'enfim',
                            'tinha', 'tínhamos', 'tinham', 'tivera', 'tivéramos', 'tenha', 
                            'tenhamos', 'tenham', 'tivesse', 'tivéssemos', 'tivessem', 'eis',
                            'tiver', 'tivermos', 'tiverem', 'terei', 'terá', 'teremos', 'tendo',
                            'terão', 'teria', 'teríamos', 'teriam', 'uso', 'fato', 'etc', 'nome',
                            'lo', 'la', 'los', 'las', 'lha', 'lho', 'lhas', 'lhos', 'lhes', 'me', 
                            'venho', 'venha', 'veio', 'gostaria', 'deles', 'delas', 'acima',
                            'mim', 'senhor', 'senhora', 'senhoras', 'senhores', 'dada', 'li',
                            'falando', 'falo', 'falou', 'falei', 'digo', 'dizia', 'dizendo',
                            'dr', 'drs', 'dra', 'sr', 'sra', 'srs', 'sras', 'exa', 'exas', 
                            'vamos', 'outro', 'outros', 'quase', 'muita', 'muitas', 'durante', 
                            'caso', 'disse', 'aparte', 'dessas', 'desses', 'vou', 'várias', 
                            'mesma', 'posso', 'possa', 'fizeram', 'faremos', 'fiz', 'farei', 
                            'fazendo', 'feitas', 'àquela', 'dela', 'dele', 'mesmos', 'horas',
                            'ocupo', 'ocupa', 'ocupar', 'ocupei', 'subo', 'assomo', 'assumo', 
                            'volto', 'volta', 'ocupamos', 'deixe', 'deixar', 'deixo', 'nobre',
                            'nessas', 'nesses', 'devemos', 'podemos', 'pudesse', 'possamos', 'possam', 
                            'possui', 'poderia', 'poderão', 'serem', 'fosse', 'ficar', 'fica', 
                            'ficou', 'fico' 'deu', 'dando', 'queremos', 'queria', 'feito', 'feita', 
                            'feitas', 'feitos', 'sabemos', 'sabem', 'sabes', 'apesar', 'senão', 
                            'termos', 'deram', 'estavam', 'questões', 'coisas', 'tantas', 
                            'tantos', 'daqui', 'algum', 'alguém', 'passar', 'passo', 'modo', 
                            'tomou', 'posse', 'vice', 'agradeço', 'parabéns', 'sair',
                            'maneira', 'seguinte', 'seguida', 'seguido', 'acho', 'vimos', 'vi',
                            'vejo', 'viu', 'vejo', 'vê', 'deu', 'dê', 'dava', 'pôs', 'pro', 'pra',
                            'traz', 'trazer', 'chegar', 'chegou', 'está', 'esta', 'fala', 'vista',
                            'ir', 'iriam', 'iam', 'indo','entretanto', 'todavia', 'existir', 
                            'estava', 'existem', 'existe', 'precisam', 'precisa', 'diziam', 'dizer',
                            'dizíamos', 'falar', 'querer', 'voltarem', 'começarem', 'cerca',
                            'daí', 'gente', 'tido', 'inicialmente', 'prazer', 'prezado', 'prezados',
                            'ninguém', 'hora', 'minutos', 'trouxe', 'saudar', 'ida', 'saudá',
                            'recentemente', 'usei', 'chega', 'cabe', 'minutos', 'peça', 'peço',
                            'trouxe', 'dentre'
                            }

In [8]:
# checking new stop words list value
print('Number of stop words with new inputs: %d' % len(spacy_stopwords))

Number of stop words with new inputs: 661


In [10]:
# import df
df = pd.read_csv('amazon_clean_discourses.tsv', sep='\t', encoding='utf-8')

In [12]:
df.head(3)

,date,session,phase,discourse_link,speaker,party,coalition,state,region,original_discourse,clean_discourse
0,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,nilton capixaba,ptb,opposition,ro,north,O SR. NILTON CAPIXABA (PTB-RO. Pronuncia o seg...,"Sr. Presidente, Sras. e Srs. Deputados, na sem..."
1,2000-01-11,195.2.51.O,parliamentarian communications,https://www.camara.leg.br/internet/SitaqWeb/Te...,sérgio novais,psb,opposition,ce,northeast,O SR. SÉRGIO NOVAIS (Bloco/PSB-CE. Como Líder....,"Sr. Presidente, Sras. e Srs. Deputados, prime..."
2,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,marcos afonso,pt,opposition,ac,north,O SR. MARCOS AFONSO (PT-AC. Pronuncia o seguin...,"Sr. Presidente, Sras. e Srs. Deputados, venho..."


In [14]:
# setting it to lowercase
df["tokenized"] = df["clean_discourse"].str.lower()

In [15]:
# removing punctuation
df["tokenized"] = df["tokenized"].apply(lambda x: " ".join(tokenizer.tokenize(x)))

In [16]:
# removing numbers
df["tokenized"] = df["tokenized"].str.replace('\d+', '')

In [17]:
# removing characters that are not letters or numbers
df["tokenized"] = df["tokenized"].str.replace('[#,@,&,º,ª]','')

In [18]:
# remove words less than two char that might be noiseness
df["tokenized"] = df['tokenized'].str.findall('\w{3,}').str.join(' ')

In [19]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in spacy_stopwords])

# removing stopwords
df["tokenized"] = df["tokenized"].apply(lambda text: remove_stopwords(text))

In [21]:
# removing other frequent words that cause noiseness to our data
freq_words = {'presidente', 'presidenta', 'deputado', 'deputados', 'deputada', 
              'deputadas', 'parlamentares', 'parlamentar', 'colegas', 'colega'
              'senador', 'senadora', 'senadores', 'senadoras', 'casa', 'feira', 'nº', 'ª', 
              'principalmente', 'pronunciamento', 'solicito', 'olá', 'precisamos', 
              'realmente', 'sobretudo', 'aparte', 'caros', 'importante', 
              'importantes', 'igualmente', 'perante', 'praticamente', 
              'especificamente', 'inicialmexnte', 'infelizmente', 'especialmente',
              'particularmente', 'exmo', 'exmos', 'querido', 'querida', 'queridos', 
              'queridas', 'companheiro', 'companheiros', 'companheira', 'companheiras',
              'registrado', 'registrada', 'registro', 'brasileira', 'brasileiro',
              'brasileiras', 'brasileiros', 'municípios', 'ministro', 'ministra',
              'encontram', 'trata', 'entreguei', 'encontram', 'aconteceram', 'acontecem',
              'tocante', 'passado', 'passada', 'agradecer', 'encaminhando', 'encaminhada'
              'cumprimento', 'ouvem', 'apelo', 'responsável', 'ninguém', 'pessoa',
              'atender', 'receber', 'torno', 'aproximadamente', 'significa', 'casos',
              'parabenizo', 'parabenizar', 'parecer', 'novamente', 'registrar', 'registro',
              'comemoramos', 'últimos', 'passa', 'dissermos', 'gostaríamos', 'lembrar',
              'passou', 'observado', 'lembrei', 'aproveito', 'manifestar', 'junto',
              'tratar', 'pretendo', 'falarei', 'trarei', 'citando', 'sinto', 'cheguei',
              'guardo', 'faça', 'constar', 'deixar', 'tomei', 'encaminho', 'mesa',
              'ocupo', 'ocupar', 'ocupei', 'subo', 'assomo', 'assumo', 'volto', 'ocupamos',
              'prestar', 'solicitação', 'manifestar', 'prestar', 'recebi', 'posse',
              'parlamento', 'discurso', 'debate', 'plenário', 'bancada', 'governador',
              'prefeito', 'senado', 'nenhum', 'texto', 'matéria', 'evento', 'votação',
              'votar', 'voto', 'frente', 'município', 'atuação', 'pegar', 'mesa', 'atestado',
              'atendendo', 'apelo', 'significa', 'atender', 'torno', 'aproximadamente'
             }

In [22]:
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in freq_words])

df["tokenized"] = df["tokenized"].apply(lambda text: remove_freqwords(text))

In [23]:
df.head(3)

,date,session,phase,discourse_link,speaker,party,coalition,state,region,original_discourse,clean_discourse,tokenized
0,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,nilton capixaba,ptb,opposition,ro,north,O SR. NILTON CAPIXABA (PTB-RO. Pronuncia o seg...,"Sr. Presidente, Sras. e Srs. Deputados, na sem...",gabinete câmara diretor ceplac comissão execut...
1,2000-01-11,195.2.51.O,parliamentarian communications,https://www.camara.leg.br/internet/SitaqWeb/Te...,sérgio novais,psb,opposition,ce,northeast,O SR. SÉRGIO NOVAIS (Bloco/PSB-CE. Como Líder....,"Sr. Presidente, Sras. e Srs. Deputados, prime...",solidariedade amapá joão capiberibe psb postur...
2,2000-01-11,195.2.51.O,short address,https://www.camara.leg.br/internet/SitaqWeb/Te...,marcos afonso,pt,opposition,ac,north,O SR. MARCOS AFONSO (PT-AC. Pronuncia o seguin...,"Sr. Presidente, Sras. e Srs. Deputados, venho...",tribuna protesto rede globo televisão veiculou...


In [24]:
df.to_csv('amazon_clean_discourses.tsv', sep='\t', encoding='utf-8', index=False)